In [18]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [19]:
df = pd.read_csv('Titanic-Dataset.csv')

In [20]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


plan --->
age and embarked is missing->
will do simple impute ->
then ohe->
scaling->
feature selection->
DT

In [21]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [22]:
X_train, X_test , y_train ,y_test= train_test_split(df.drop('Survived', axis=1),
                                                    df['Survived'],
                                                    test_size=0.2,
                                                    random_state=42)

X_train.shape , X_test.shape

((712, 7), (179, 7))

In [23]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


in pipeline use index of column rather than name because it will return numpy and numpy dont have column name

In [24]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [25]:
#one hot encoding 

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [26]:
#scaling

trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [27]:
# feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [28]:
#train the model

trf5 = DecisionTreeClassifier()

In [49]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

alternative make_pipeline -> use it . It is more simple

In [30]:
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [36]:
pipe.fit(X_train,y_train) 
#if not adding model in the pipeline then fit_transform . 

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                      

In [37]:
from sklearn import set_config
set_config(display='diagram')#use for show the tree diagram but default not needed

In [42]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.49884615])

In [43]:
y_pred = pipe.predict(X_test)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

Cross validation using cross val_score

In [45]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5, scoring='accuracy').mean()

0.6391214419383433

GridSearch using Pipeline || Hyper parameter tuning

In [50]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [51]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params ,cv =5 , scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x15a84d630>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [52]:
grid.best_score_

0.6391214419383433

In [53]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))